# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 1 2022 11:40AM,243149,10,PRF-34301,Bio-PRF,Released
1,Aug 1 2022 11:40AM,243149,10,PRF-34358,Bio-PRF,Released
2,Aug 1 2022 11:40AM,243149,10,PRF-34361,Bio-PRF,Released
3,Aug 1 2022 11:40AM,243149,10,PRF-34366,Bio-PRF,Released
4,Aug 1 2022 11:40AM,243149,10,PRF-34383,Bio-PRF,Released
5,Aug 1 2022 11:40AM,243149,10,PRF-34397,Bio-PRF,Released
6,Aug 1 2022 11:40AM,243149,10,6787446,Bio-PRF,Released
7,Aug 1 2022 11:38AM,243148,10,8339173,Eywa Pharma Inc.,Released
8,Aug 1 2022 11:38AM,243148,10,8339174,Eywa Pharma Inc.,Released
9,Aug 1 2022 11:38AM,243148,10,8339192,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,243149,Released,7
32,243150,Released,49
33,243152,Released,2
34,243153,Released,29
35,243154,Released,20


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243149,NaN,NaN,7.0
243150,NaN,NaN,49.0
243152,NaN,NaN,2.0
243153,NaN,NaN,29.0
243154,NaN,NaN,20.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243085,0.0,0.0,1.0
243091,0.0,0.0,1.0
243101,0.0,0.0,1.0
243108,13.0,13.0,6.0
243110,0.0,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243085,0,0,1
243091,0,0,1
243101,0,0,1
243108,13,13,6
243110,0,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243085,0,0,1
1,243091,0,0,1
2,243101,0,0,1
3,243108,13,13,6
4,243110,0,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243085,,,1
1,243091,,,1
2,243101,,,1
3,243108,13,13,6
4,243110,,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 1 2022 11:40AM,243149,10,Bio-PRF
7,Aug 1 2022 11:38AM,243148,10,Eywa Pharma Inc.
30,Aug 1 2022 11:35AM,243154,15,"Alliance Pharma, Inc."
50,Aug 1 2022 11:30AM,243153,10,ISDIN Corporation
79,Aug 1 2022 11:29AM,243152,19,"AdvaGen Pharma, Ltd"
81,Aug 1 2022 11:22AM,243150,10,ISDIN Corporation
130,Aug 1 2022 11:13AM,243133,15,"Brookfield Pharmaceuticals, LLC"
156,Aug 1 2022 11:13AM,243134,15,"Carwin Pharmaceutical Associates, LLC"
158,Aug 1 2022 11:12AM,243147,20,"ACI Healthcare USA, Inc."
164,Aug 1 2022 11:07AM,243146,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 1 2022 11:40AM,243149,10,Bio-PRF,,,7
1,Aug 1 2022 11:38AM,243148,10,Eywa Pharma Inc.,,,23
2,Aug 1 2022 11:35AM,243154,15,"Alliance Pharma, Inc.",,,20
3,Aug 1 2022 11:30AM,243153,10,ISDIN Corporation,,,29
4,Aug 1 2022 11:29AM,243152,19,"AdvaGen Pharma, Ltd",,,2
5,Aug 1 2022 11:22AM,243150,10,ISDIN Corporation,,,49
6,Aug 1 2022 11:13AM,243133,15,"Brookfield Pharmaceuticals, LLC",,,26
7,Aug 1 2022 11:13AM,243134,15,"Carwin Pharmaceutical Associates, LLC",,,2
8,Aug 1 2022 11:12AM,243147,20,"ACI Healthcare USA, Inc.",,,6
9,Aug 1 2022 11:07AM,243146,19,"GCH Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 11:40AM,243149,10,Bio-PRF,7,,
1,Aug 1 2022 11:38AM,243148,10,Eywa Pharma Inc.,23,,
2,Aug 1 2022 11:35AM,243154,15,"Alliance Pharma, Inc.",20,,
3,Aug 1 2022 11:30AM,243153,10,ISDIN Corporation,29,,
4,Aug 1 2022 11:29AM,243152,19,"AdvaGen Pharma, Ltd",2,,
5,Aug 1 2022 11:22AM,243150,10,ISDIN Corporation,49,,
6,Aug 1 2022 11:13AM,243133,15,"Brookfield Pharmaceuticals, LLC",26,,
7,Aug 1 2022 11:13AM,243134,15,"Carwin Pharmaceutical Associates, LLC",2,,
8,Aug 1 2022 11:12AM,243147,20,"ACI Healthcare USA, Inc.",6,,
9,Aug 1 2022 11:07AM,243146,19,"GCH Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 11:40AM,243149,10,Bio-PRF,7,,
1,Aug 1 2022 11:38AM,243148,10,Eywa Pharma Inc.,23,,
2,Aug 1 2022 11:35AM,243154,15,"Alliance Pharma, Inc.",20,,
3,Aug 1 2022 11:30AM,243153,10,ISDIN Corporation,29,,
4,Aug 1 2022 11:29AM,243152,19,"AdvaGen Pharma, Ltd",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 11:40AM,243149,10,Bio-PRF,7.0,NaN,NaN
1,Aug 1 2022 11:38AM,243148,10,Eywa Pharma Inc.,23.0,NaN,NaN
2,Aug 1 2022 11:35AM,243154,15,"Alliance Pharma, Inc.",20.0,NaN,NaN
3,Aug 1 2022 11:30AM,243153,10,ISDIN Corporation,29.0,NaN,NaN
4,Aug 1 2022 11:29AM,243152,19,"AdvaGen Pharma, Ltd",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 1 2022 11:40AM,243149,10,Bio-PRF,7.0,0.0,0.0
1,Aug 1 2022 11:38AM,243148,10,Eywa Pharma Inc.,23.0,0.0,0.0
2,Aug 1 2022 11:35AM,243154,15,"Alliance Pharma, Inc.",20.0,0.0,0.0
3,Aug 1 2022 11:30AM,243153,10,ISDIN Corporation,29.0,0.0,0.0
4,Aug 1 2022 11:29AM,243152,19,"AdvaGen Pharma, Ltd",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4133240,374.0,10.0,1.0
15,729421,48.0,0.0,0.0
16,243123,2.0,0.0,0.0
19,1458803,11.0,14.0,13.0
20,1215569,57.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4133240,374.0,10.0,1.0
1,15,729421,48.0,0.0,0.0
2,16,243123,2.0,0.0,0.0
3,19,1458803,11.0,14.0,13.0
4,20,1215569,57.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,374.0,10.0,1.0
1,15,48.0,0.0,0.0
2,16,2.0,0.0,0.0
3,19,11.0,14.0,13.0
4,20,57.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,374.0
1,15,Released,48.0
2,16,Released,2.0
3,19,Released,11.0
4,20,Released,57.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Completed,1.0,0.0,0.0,13.0,0.0
Executing,10.0,0.0,0.0,14.0,0.0
Released,374.0,48.0,2.0,11.0,57.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Completed,1.0,0.0,0.0,13.0,0.0
1,Executing,10.0,0.0,0.0,14.0,0.0
2,Released,374.0,48.0,2.0,11.0,57.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Completed,1.0,0.0,0.0,13.0,0.0
1,Executing,10.0,0.0,0.0,14.0,0.0
2,Released,374.0,48.0,2.0,11.0,57.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()